In [2]:
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import models, transforms
import os

BATCH_SIZE=32
EPOCH=100
LEARNING_RATE=0.001

class MyDataset(Dataset):
    def __init__(self, data_dir, transform=None):

        self.class2id = {'MildDemented': 0, 'ModerateDemented': 1, 'NonDemented': 2, 'VeryMildDemented': 3}
        self.data = self.get_img(data_dir)
        self.transform = transform
 
    def __getitem__(self, index):
        path, label = self.data[index]
        img = Image.open(path)
        if self.transform is not None:
            img = self.transform(img)
 
        return img, self.class2id[label]
 
    def __len__(self):
        return len(self.data)
 
    @staticmethod
    def get_img(data_dir):
        all_path = []
        labels = os.listdir(data_dir)
        for label in labels:    
            filepath = data_dir + "/" + label
            filename  = os.listdir(filepath)
            for fname in filename:
                ffpath = filepath + "/" + fname
                path = [ffpath, label]
                all_path.append(path)
 
        return all_path
    
data_transforms = {
    'train': transforms.Compose([
        transforms.CenterCrop([180, 150]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(0.1),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
        transforms.Normalize([0.2582], [0.1033])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop([180, 150]),
        transforms.ToTensor(),
        transforms.Normalize([0.2582], [0.1033])
    ]),
}

data_dir = 'output'
datasets = {x: MyDataset(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']}

dataloaders = {
    'train': DataLoader(datasets['train'], batch_size=BATCH_SIZE,
                                            shuffle=True),
    'val': DataLoader(datasets['val'], batch_size=BATCH_SIZE,
                                            shuffle=True)
                                            }

dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}
class2id = datasets['train'].class2id
print(dataset_sizes)
data, labels = next(iter(dataloaders['train']))
print(data.shape)

{'train': 4095, 'val': 1026}
torch.Size([32, 1, 180, 150])


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import models, transforms
from torchmetrics.functional import auroc
import io
import matplotlib.pyplot as plt
import time
import os
import copy
from glob import glob
from tqdm import tqdm

import warnings
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score,recall_score,roc_auc_score,roc_curve
warnings.simplefilter('ignore')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [15]:
train_losses = []
train_acc = []
val_losses = []
val_acc = []

def train_model(model, criterion, optimizer, name, num_epochs=40):

    #Creating a folder to save the model performance.
    try:
        os.mkdir(f'./{name}')
    except:
        print('existed')

    since = time.time()

    best_acc = 0.0

    for epoch in range(num_epochs):

        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            total_loss = 0.0
            total_corrects = 0
            #epochs
            
            i = int(len(datasets[phase]) / BATCH_SIZE)
            
            for _ in tqdm(range(i)):
                #Loading Data

                inputs, labels = next(iter(dataloaders[phase]))
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                total_loss += loss.item() * labels.size(0)
                total_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = total_loss / dataset_sizes[phase]
            epoch_acc = total_corrects.double() / dataset_sizes[phase]

            #AUC: {:.4f} , epoch_auc
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), './{}/best_model_{:.4f}acc_{}epochs.pth'.format(name, epoch_acc, epoch))
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_losses.append(epoch_loss)
                val_acc.append(epoch_acc)



    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


### Model

In [4]:
class Myres50(nn.Module):
	def __init__(self, input_ch, num_class, weights=True):
		super(Myres50, self).__init__()
		# self.model = models.resnet50(weights)
		self.model = models.resnext50_32x4d(weights)
		conv1_weight = torch.mean(self.model.conv1.weight, dim=1, keepdim=True).repeat(1,input_ch,1,1)#取出从conv1权重并进行平均和拓展
		conv1 = nn.Conv2d(input_ch, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False) #新的conv1层
		model_dict = self.model.state_dict()#获取整个网络的预训练权重
		self.model.conv1 = conv1 #替换原来的conv1
		model_dict['conv1.weight'] = conv1_weight #将conv1权重替换为新conv1权重
		model_dict.update(model_dict)#更新整个网络的预训练权重
		self.model.load_state_dict(model_dict)#载入新预训练权重
		
		fcif = self.model.fc.in_features
		self.model.fc =nn.Sequential(
			nn.Linear(fcif, 2048),
			nn.BatchNorm1d(2048),
			nn.ReLU(True),
			nn.Dropout(0.5),
			nn.Linear(2048, 1024),
			nn.BatchNorm1d(1024),
			nn.ReLU(True),
			nn.Dropout(0.5),
			nn.Linear(1024, num_class),
			nn.Softmax()
			)
		
	def forward(self,x):
		x = self.model(x)
		return x

class Mydense121(nn.Module):
	def __init__(self, input_ch, num_class, weights=True):
		super(Mydense121, self).__init__()
		self.model = models.densenet121(weights)
		features_conv0_weight = torch.mean(self.model.features.conv0.weight, dim=1, keepdim=True).repeat(1,input_ch,1,1)#取出从conv1权重并进行平均和拓展
		features_conv0 = nn.Conv2d(input_ch, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False) #新的conv1层
		model_dict = self.model.state_dict()#获取整个网络的预训练权重
		self.model.features.conv0 = features_conv0 #替换原来的conv1
		model_dict['features.conv0.weight'] = features_conv0_weight #将conv1权重替换为新conv1权重
		model_dict.update(model_dict)#更新整个网络的预训练权重
		self.model.load_state_dict(model_dict)#载入新预训练权重
		
		cif = self.model.classifier.in_features
		self.model.fc =nn.Sequential(
			nn.Linear(cif, 2048),
			nn.BatchNorm1d(2048),
			nn.ReLU(True),
			nn.Dropout(0.5),
			nn.Linear(2048, 1024),
			nn.BatchNorm1d(1024),
			nn.ReLU(True),
			nn.Dropout(0.5),
			nn.Linear(1024, num_class),
			nn.Softmax()
			)
		
	def forward(self,x):
		x = self.model(x)
		return x

model = Mydense121(input_ch=1, num_class=len(class2id), weights=True)
name = model._get_name()
go = 1 #继续上次的训练
if go:
	model.load_state_dict(torch.load(f'./{name}/'+sorted(os.listdir(f'./{name}'))[-1]))
print(model)

Mydense121(
  (model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.

### Train

In [22]:
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_model(model, criterion, optimizer, name, num_epochs=EPOCH)

existed
Epoch 1/100
----------


 38%|███▊      | 12/32 [00:01<00:02,  7.78it/s]


KeyboardInterrupt: 

### Test

In [10]:
best_model = Mydense121(input_ch=1, num_class=len(class2id), weights=True)
# best_model.load_state_dict(torch.load(f'./{name}/'+sorted(os.listdir(f'./{name}'))[-1]))
best_model.load_state_dict(torch.load("Mydense121/best_model_0.9805acc_99epochs.pth"))
best_model = best_model.to(device)

In [7]:
test_transforms = transforms.Compose([
        transforms.CenterCrop([180, 150]),
        transforms.ToTensor(),
        transforms.Normalize([0.2582], [0.1033])
    ])

In [24]:
test_dir = 'Alzheimer_s Dataset/test'
test_set = MyDataset(test_dir, test_transforms)

test_loader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle=False)

def test_model(model, device, test_loader):
    total_preds = []
    total_labels = []
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs, 1)
            total_preds.append(predicted.cpu())
            total_labels.append(labels.cpu())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(total_labels, total_preds)
    print(roc_auc_score(total_labels, total_preds))
    print('Correct Prediction: {:d}  Total Images: {:d}'.format(correct, total))
    print('Test Accuracy = {:f}'.format(correct / total))

test_model(best_model, device, test_loader)

#dense121  train:0.997558   val:0.983431    test:0.768569

AttributeError: 'list' object has no attribute 'squeeze'